# American Community Survey 2015-5yr Estimates 

In order to relate and compare to the hotspot data, we have the American Community Survey (ACS) data, 5yr estimates for 2015. Our hot spot data is covering 2014-2019, so the 5yr estimate covers that range nicely. Moreover, the individual year ACS data did not have zipcode level data. 

This subset of the ACS report describes the percentages of the population with a certain income level. This data will enrich our hotspot analysis to illuminate insights about recent solar panel adoption and income.


## Data Cleaning

In [24]:
## IMPORTS
import numpy as np
import pandas as pd
import arcpy 
import re
arcpy.env.workspace = "C:/Users/Perso/Desktop/Tulane U/Junior Year/Semester 6/EENS 4030- Advanced GIS/FINAL PROJECT/Dataset"
%cd "C:/Users/Perso/Desktop/Tulane U/Junior Year/Semester 6/EENS 4030- Advanced GIS/FINAL PROJECT/Dataset/ACS"


C:\Users\Perso\Desktop\Tulane U\Junior Year\Semester 6\EENS 4030- Advanced GIS\FINAL PROJECT\Dataset\ACS


In [25]:
acs2015 = pd.read_csv("ACS-2015-5yr-Column-wise.csv", header= 1)

acs2015.head(5)

,Geography,Geographic Area Name,Households!!Estimate!!Total,Households!!Margin of Error!!Total,Annotation of Households!!Margin of Error!!Total,Annotation of Households!!Estimate!!Total,Families!!Estimate!!Total,Families!!Margin of Error!!Total,Annotation of Families!!Margin of Error!!Total,Annotation of Families!!Estimate!!Total,Married-couple families!!Estimate!!Total,Married-couple families!!Margin of Error!!Total,Annotation of Married-couple families!!Margin of Error!!Total,Annotation of Married-couple families!!Estimate!!Total,Nonfamily households!!Estimate!!Total,Annotation of Nonfamily households!!Estimate!!Total,Nonfamily households!!Margin of Error!!Total,Annotation of Nonfamily households!!Margin of Error!!Total,"Households!!Estimate!!Less than $10,000","Households!!Margin of Error!!Less than $10,000","Annotation of Households!!Margin of Error!!Less than $10,000","Annotation of Households!!Estimate!!Less than $10,000","Families!!Estimate!!Less than $10,000","Annotation of Families!!Estimate!!Less than $10,000","Families!!Margin of Error!!Less than $10,000","Annotation of Families!!Margin of Error!!Less than $10,000","Married-couple families!!Estimate!!Less than $10,000","Married-couple families!!Margin of Error!!Less than $10,000","Annotation of Married-couple families!!Margin of Error!!Less than $10,000","Annotation of Married-couple families!!Estimate!!Less than $10,000","Nonfamily households!!Estimate!!Less than $10,000","Nonfamily households!!Margin of Error!!Less than $10,000","Annotation of Nonfamily households!!Margin of Error!!Less than $10,000","Annotation of Nonfamily households!!Estimate!!Less than $10,000","Households!!Estimate!!$10,000 to $14,999","Households!!Margin of Error!!$10,000 to $14,999","Annotation of Households!!Margin of Error!!$10,000 to $14,999","Annotation of Households!!Estimate!!$10,000 to $14,999","Families!!Estimate!!$10,000 to $14,999","Annotation of Families!!Estimate!!$10,000 to $14,999",...,Annotation of Married-couple families!!Estimate!!PERCENT IMPUTED!!Household income in the past 12 months,Married-couple families!!Margin of Error!!PERCENT IMPUTED!!Household income in the past 12 months,Annotation of Married-couple families!!Margin of Error!!PERCENT IMPUTED!!Household income in the past 12 months,Nonfamily households!!Estimate!!PERCENT IMPUTED!!Household income in the past 12 months,Annotation of Nonfamily households!!Estimate!!PERCENT IMPUTED!!Household income in the past 12 months,Nonfamily households!!Margin of Error!!PERCENT IMPUTED!!Household income in the past 12 months,Annotation of Nonfamily households!!Margin of Error!!PERCENT IMPUTED!!Household income in the past 12 months,Households!!Estimate!!PERCENT IMPUTED!!Family income in the past 12 months,Households!!Margin of Error!!PERCENT IMPUTED!!Family income in the past 12 months,Annotation of Households!!Margin of Error!!PERCENT IMPUTED!!Family income in the past 12 months,Annotation of Households!!Estimate!!PERCENT IMPUTED!!Family income in the past 12 months,Families!!Estimate!!PERCENT IMPUTED!!Family income in the past 12 months,Families!!Margin of Error!!PERCENT IMPUTED!!Family income in the past 12 months,Annotation of Families!!Margin of Error!!PERCENT IMPUTED!!Family income in the past 12 months,Annotation of Families!!Estimate!!PERCENT IMPUTED!!Family income in the past 12 months,Married-couple families!!Estimate!!PERCENT IMPUTED!!Family income in the past 12 months,Married-couple families!!Margin of Error!!PERCENT IMPUTED!!Family income in the past 12 months,Annotation of Married-couple families!!Margin of Error!!PERCENT IMPUTED!!Family income in the past 12 months,Annotation of Married-couple families!!Estimate!!PERCENT IMPUTED!!Family income in the past 12 months,Nonfamily households!!Estimate!!PERCENT IMPUTED!!Family income in the past 12 months,Annotation of Nonfamily households!!Estimate!!PERCENT IMPUTED!!Family income in the past 12 months,Nonfamily households!!Margin of Error!!PERCENT IMPUTED!!Family inco

#### Delete all margins of error & annotations

In [26]:
drops = ['Margin',
'Annotation',
'PERCENT IMPUTED']

for word in drops:
    acs2015 = acs2015[acs2015.columns.drop(list(acs2015.filter(regex= word)))]
    

acs2015.head(5)

,Geography,Geographic Area Name,Households!!Estimate!!Total,Families!!Estimate!!Total,Married-couple families!!Estimate!!Total,Nonfamily households!!Estimate!!Total,"Households!!Estimate!!Less than $10,000","Families!!Estimate!!Less than $10,000","Married-couple families!!Estimate!!Less than $10,000","Nonfamily households!!Estimate!!Less than $10,000","Households!!Estimate!!$10,000 to $14,999","Families!!Estimate!!$10,000 to $14,999","Married-couple families!!Estimate!!$10,000 to $14,999","Nonfamily households!!Estimate!!$10,000 to $14,999","Households!!Estimate!!$15,000 to $24,999","Families!!Estimate!!$15,000 to $24,999","Married-couple families!!Estimate!!$15,000 to $24,999","Nonfamily households!!Estimate!!$15,000 to $24,999","Households!!Estimate!!$25,000 to $34,999","Families!!Estimate!!$25,000 to $34,999","Married-couple families!!Estimate!!$25,000 to $34,999","Nonfamily households!!Estimate!!$25,000 to $34,999","Households!!Estimate!!$35,000 to $49,999","Families!!Estimate!!$35,000 to $49,999","Married-couple families!!Estimate!!$35,000 to $49,999","Nonfamily households!!Estimate!!$35,000 to $49,999","Households!!Estimate!!$50,000 to $74,999","Families!!Estimate!!$50,000 to $74,999","Married-couple families!!Estimate!!$50,000 to $74,999","Nonfamily households!!Estimate!!$50,000 to $74,999","Households!!Estimate!!$75,000 to $99,999","Families!!Estimate!!$75,000 to $99,999","Married-couple families!!Estimate!!$75,000 to $99,999","Nonfamily households!!Estimate!!$75,000 to $99,999","Households!!Estimate!!$100,000 to $149,999","Families!!Estimate!!$100,000 to $149,999","Married-couple families!!Estimate!!$100,000 to $149,999","Nonfamily households!!Estimate!!$100,000 to $149,999","Households!!Estimate!!$150,000 to $199,999","Families!!Estimate!!$150,000 to $199,999","Married-couple families!!Estimate!!$150,000 to $199,999","Nonfamily households!!Estimate!!$150,000 to $199,999","Households!!Estimate!!$200,000 or more","Families!!Estimate!!$200,000 or more","Married-couple families!!Estimate!!$200,000 or more","Nonfamily households!!Estimate!!$200,000 or more",Households!!Estimate!!Median income (dollars),Families!!Estimate!!Median income (dollars),Married-couple families!!Estimate!!Median income (dollars),Nonfamily households!!Estimate!!Median income (dollars),Households!!Estimate!!Mean income (dollars),Families!!Estimate!!Mean income (dollars),Married-couple families!!Estimate!!Mean income (dollars),Nonfamily households!!Estimate!!Mean income (dollars),Unnamed: 258
0,8600000US89010,ZCTA5 89010,252,154,117,98,11.1,9.1,0.0,14.3,0.0,5.8,0.0,0.0,14.7,6.5,5.1,18.4,23.4,24.7,26.5,21.4,16.7,12.3,16.2,23.5,17.9,14.9,17.1,22.4,4.0,6.5,8.5,0.0,12.3,20.1,26.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35625,36875,(X),31833,46479,54045,N,34140,NaN
1,8600000US89019,ZCTA5 89019,861,513,317,348,0.0,0.0,0.0,0.0,5.9,0.0,0.0,14.7,23.5,3.7,6.0,52.6,12.4,12.9,14.5,14.9,27.8,40.7,12.6,5.5,10.1,11.1,15.8,8.6,8.2,13.8,22.4,0.0,7.4,9.9,16.1,3.7,4.6,7.8,12.6,0.0,0.0,0.0,0.0,0.0,38134,42083,75154,21316,50325,64717,N,28528,NaN
2,8600000US89060,ZCTA5 89060,4011,2340,1889,1671,4.7,2.6,1.5,8.6,10.3,5.6,1.6,16.8,17.2,13.3,11.8,21.5,10.5,8.7,7.9,13.0,22.0,25.1,28.3,17.7,17.9,20.5,22.1,17.1,9.1,12.5,12.3,3.5,7.6,10.3,12.8,1.8,0.4,0.7,0.8,0.0,0.3,0.6,0.7,0.0,40700,45808,(X),28371,47436,56329,N,33527,NaN
3,8600000US89061,ZCTA5 89061,2214,1573,1383,641,3.2,2.2,1.1,5.5,3.6,1.3,1.4,9.4,8.1,6.3,3.4,12.5,13.3,9.3,10.6,22.9,13.6,14.6,15.5,11.2,26.3,29.8,31.8,17.6,9.6,12.5,13.2,2.5,13.9,16.4,17.5,7.6,6.5,4.7,2.2,10.8,2.0,2.9,3.3,0.0,54708,62292,(X),29886,69133,75494,N,52743,NaN
4,8600000US89439,ZCTA5 89439,724,473,374,251,3.9,0.0,0.0,11.2,4.8,3.6,0.0,7.2,3.3,5.1,4.5,0.0,14.6,7.0,8.8,29.1,18.1,11.0,0.0,31.5,16.7,22.0,26.2,6.8,8.3,5.1,6.4,14.3,13.7,20.9,21.9,0.0,0.4,0.6,0.8,0.0,16.2,24.7,31.3,0.0,57500,95197,(X),38516,149584,208321,N,38537,NaN


Make the zipcode into an integer

In [27]:
acs2015["Geographic Area Name"] = acs2015["Geographic Area Name"].apply(lambda x: int(re.sub("ZCTA5", "", x)))


### Simplification of Income Brackets & Households

For the purpose of simplifying income brackets for analysis, I choose to create three primary income brackets: 
* High Income (100k ->)
* Middle Income (50k -> 100k)
* Low Income (0k -> 50k)

I also decided to only look at households.

In [28]:
geos = acs2015.loc[:, ('Geography', 'Geographic Area Name')]

## Filter out non-households
acs2015_filtered = acs2015.filter(regex= 'Households')

acs2015_filtered.head(5)

,Households!!Estimate!!Total,"Households!!Estimate!!Less than $10,000","Households!!Estimate!!$10,000 to $14,999","Households!!Estimate!!$15,000 to $24,999","Households!!Estimate!!$25,000 to $34,999","Households!!Estimate!!$35,000 to $49,999","Households!!Estimate!!$50,000 to $74,999","Households!!Estimate!!$75,000 to $99,999","Households!!Estimate!!$100,000 to $149,999","Households!!Estimate!!$150,000 to $199,999","Households!!Estimate!!$200,000 or more",Households!!Estimate!!Median income (dollars),Households!!Estimate!!Mean income (dollars)
0,252,11.1,0.0,14.7,23.4,16.7,17.9,4.0,12.3,0.0,0.0,35625,46479
1,861,0.0,5.9,23.5,12.4,27.8,10.1,8.2,7.4,4.6,0.0,38134,50325
2,4011,4.7,10.3,17.2,10.5,22.0,17.9,9.1,7.6,0.4,0.3,40700,47436
3,2214,3.2,3.6,8.1,13.3,13.6,26.3,9.6,13.9,6.5,2.0,54708,69133
4,724,3.9,4.8,3.3,14.6,18.1,16.7,8.3,13.7,0.4,16.2,57500,149584


In [29]:
## Renaming the households label

acs2015_filtered.columns = list(map(lambda x: x.replace("Households!!Estimate!!", ''), acs2015_filtered.columns))
acs2015_filtered.head(5)

,Total,"Less than $10,000","$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 or more",Median income (dollars),Mean income (dollars)
0,252,11.1,0.0,14.7,23.4,16.7,17.9,4.0,12.3,0.0,0.0,35625,46479
1,861,0.0,5.9,23.5,12.4,27.8,10.1,8.2,7.4,4.6,0.0,38134,50325
2,4011,4.7,10.3,17.2,10.5,22.0,17.9,9.1,7.6,0.4,0.3,40700,47436
3,2214,3.2,3.6,8.1,13.3,13.6,26.3,9.6,13.9,6.5,2.0,54708,69133
4,724,3.9,4.8,3.3,14.6,18.1,16.7,8.3,13.7,0.4,16.2,57500,149584


In [30]:
## Notice non-numerical values (X), N, and '-'

display(acs2015_filtered.iloc[358])
display(acs2015_filtered.iloc[66])

Total                       144
Less than $10,000          46.5
$10,000 to $14,999          0.0
$15,000 to $24,999          0.0
$25,000 to $34,999          0.0
$35,000 to $49,999         18.8
$50,000 to $74,999         34.7
$75,000 to $99,999          0.0
$100,000 to $149,999        0.0
$150,000 to $199,999        0.0
$200,000 or more            0.0
Median income (dollars)     (X)
Mean income (dollars)         N
Name: 358, dtype: object

Total                      0
Less than $10,000          -
$10,000 to $14,999         -
$15,000 to $24,999         -
$25,000 to $34,999         -
$35,000 to $49,999         -
$50,000 to $74,999         -
$75,000 to $99,999         -
$100,000 to $149,999       -
$150,000 to $199,999       -
$200,000 or more           -
Median income (dollars)    -
Mean income (dollars)      -
Name: 66, dtype: object

#### Filtering out the non-numerical data
From the file metadata:

Explanation of Symbols:<ol><li>An ''**'' entry in the margin of error column indicates that either no sample observations or too few sample observations were available to compute a standard error and thus the margin of error. A statistical test is not appropriate.</li>
<li>An ''-'' entry in the estimate column indicates that either no sample observations or too few sample observations were available to compute an estimate, or a ratio of medians cannot be calculated because one or both of the median estimates falls in the lowest interval or upper interval of an open-ended distribution.</li>
<li>An ''-'' following a median estimate means the median falls in the lowest interval of an open-ended distribution.</li>
<li>An ''+'' following a median estimate means the median falls in the upper interval of an open-ended distribution.</li>
<li>An ''***'' entry in the margin of error column indicates that the median falls in the lowest interval or upper interval of an open-ended distribution. A statistical test is not appropriate.</li>
<li>An ''*****'' entry in the margin of error column indicates that the estimate is controlled. A statistical test for sampling variability is not appropriate. </li>
<li>An ''N'' entry in the estimate and margin of error columns indicates that data for this geographic area cannot be displayed because the number of sample cases is too small.</li>
<li>An ''(X)'' means that the estimate is not applicable or not available.</li></ol><br /><br />


In [31]:
## pd.eval evaluates the expression along all axes of the df.
## DataFrame.any check that any values in the row is true.
## DataFrame.mask assigns NaN values to rows which are True (assigned True from the eval/any)
## DataFrame.dropna drops any rows with NaN values

symbols= ['**','-','+','***', '*****', 'N', '(X)']

for letter in symbols:
    acs2015_filtered = acs2015_filtered.mask(pd.eval("acs2015_filtered == letter").any(axis=1)).dropna()


acs2015_filtered

,Total,"Less than $10,000","$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 or more",Median income (dollars),Mean income (dollars)
0,252.0,11.1,0.0,14.7,23.4,16.7,17.9,4.0,12.3,0.0,0.0,35625,46479
1,861.0,0.0,5.9,23.5,12.4,27.8,10.1,8.2,7.4,4.6,0.0,38134,50325
2,4011.0,4.7,10.3,17.2,10.5,22.0,17.9,9.1,7.6,0.4,0.3,40700,47436
3,2214.0,3.2,3.6,8.1,13.3,13.6,26.3,9.6,13.9,6.5,2.0,54708,69133
4,724.0,3.9,4.8,3.3,14.6,18.1,16.7,8.3,13.7,0.4,16.2,57500,149584
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1763,386.0,3.4,12.7,16.8,3.4,10.9,20.7,12.4,7.0,7.5,5.2,51964,70068
1764,272.0,11.8,5.1,3.7,8.5,24.3,16.2,11.8,14.0,4.8,0.0,41875,59204
1765,11282.0,5.8,6.9,10.7,15.0,12.0,18.1,10.8,12.5,5.5,2.7,49293,65812
1767,7207.0,3.5,2.4,4.9,7.8,10.5,17.7,18.1,17.6,7.6,10.1,78206,106428


In [32]:
## Now we should only have numerical strings.
## So, we can now convert them to floats.
for col in acs2015_filtered.columns:
    acs2015_filtered[col] = pd.to_numeric(acs2015_filtered[col], errors= 'ignore')

acs2015_filtered.dtypes

Total                      float64
Less than $10,000          float64
$10,000 to $14,999         float64
$15,000 to $24,999         float64
$25,000 to $34,999         float64
$35,000 to $49,999         float64
$50,000 to $74,999         float64
$75,000 to $99,999         float64
$100,000 to $149,999       float64
$150,000 to $199,999       float64
$200,000 or more           float64
Median income (dollars)     object
Mean income (dollars)        int64
dtype: object

In [33]:
acs2015_filtered.iloc[1630]

Total                          62.0
Less than $10,000               0.0
$10,000 to $14,999              0.0
$15,000 to $24,999              0.0
$25,000 to $34,999              0.0
$35,000 to $49,999             17.7
$50,000 to $74,999              0.0
$75,000 to $99,999              0.0
$100,000 to $149,999           11.3
$150,000 to $199,999            0.0
$200,000 or more               71.0
Median income (dollars)    250,000+
Mean income (dollars)        206594
Name: 1750, dtype: object

In [34]:
acs2015_filtered['Geography'] = geos['Geography']
acs2015_filtered['Geographic Area Name'] = geos['Geographic Area Name']

acs2015_filtered

,Total,"Less than $10,000","$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 or more",Median income (dollars),Mean income (dollars),Geography,Geographic Area Name
0,252.0,11.1,0.0,14.7,23.4,16.7,17.9,4.0,12.3,0.0,0.0,35625,46479,8600000US89010,89010
1,861.0,0.0,5.9,23.5,12.4,27.8,10.1,8.2,7.4,4.6,0.0,38134,50325,8600000US89019,89019
2,4011.0,4.7,10.3,17.2,10.5,22.0,17.9,9.1,7.6,0.4,0.3,40700,47436,8600000US89060,89060
3,2214.0,3.2,3.6,8.1,13.3,13.6,26.3,9.6,13.9,6.5,2.0,54708,69133,8600000US89061,89061
4,724.0,3.9,4.8,3.3,14.6,18.1,16.7,8.3,13.7,0.4,16.2,57500,149584,8600000US89439,89439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1763,386.0,3.4,12.7,16.8,3.4,10.9,20.7,12.4,7.0,7.5,5.2,51964,70068,8600000US96146,96146
1764,272.0,11.8,5.1,3.7,8.5,24.3,16.2,11.8,14.0,4.8,0.0,41875,59204,8600000US96148,96148
1765,11282.0,5.8,6.9,10.7,15.0,12.0,18.1,10.8,12.5,5.5,2.7,49293,65812,8600000US96150,96150
1767,7207.0,3.5,2.4,4.9,7.8,10.5,17.7,18.1,17.6,7.6,10.1,78206,106428,8600000US96161,96161


#### Merging income brackets
For the purpose of simplifying income brackets for analysis, the three primary income brackets: 
* High Income (100k ->)
* Middle Income (35k -> 100k)
* Low Income (0k -> 35k)


In [35]:
high_income = ['$100,000 to $149,999', '$150,000 to $199,999', '$200,000 or more']

middle_income = ['$35,000 to $49,999', '$50,000 to $74,999', '$75,000 to $99,999']

low_income = ['Less than $10,000', '$10,000 to $14,999', '$15,000 to $24,999',
              '$25,000 to $34,999' ]




In [43]:
## Using the ArcGIS field calculator
acs2015_filtered["Low income"] = pd.eval("acs2015_filtered[low_income[0]] + acs2015_filtered[low_income[1]] + acs2015_filtered[low_income[2]]+ acs2015_filtered[low_income[3]] ", engine= 'python')
acs2015_filtered["Middle income"] =  pd.eval("acs2015_filtered[middle_income[0]] + acs2015_filtered[middle_income[1]] + acs2015_filtered[middle_income[2]]", engine= 'python')
acs2015_filtered["High income"] = pd.eval("acs2015_filtered[high_income[0]] + acs2015_filtered[high_income[1]] + acs2015_filtered[high_income[2]]", engine= 'python')


#### Verify & Export

In [44]:
## Verify that the values are approximately 100%.
## There will always be a small amount of +- due to conversions and floats etc.

pd.eval("acs2015_filtered['Low income'] + acs2015_filtered['Middle income'] + acs2015_filtered['High income']", engine= 'python').describe()

## I'm happy with the bounds 99.7 - 100.3

count    1646.000000
mean      100.000243
std         0.090456
min        99.700000
25%        99.900000
50%       100.000000
75%       100.100000
max       100.300000
dtype: float64

In [45]:
## Final Visual Check
acs2015_filtered

,Total,"Less than $10,000","$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 or more",Median income (dollars),Mean income (dollars),Geography,Geographic Area Name,Low income,Middle income,High income
0,252.0,11.1,0.0,14.7,23.4,16.7,17.9,4.0,12.3,0.0,0.0,35625,46479,8600000US89010,89010,49.2,38.6,12.3
1,861.0,0.0,5.9,23.5,12.4,27.8,10.1,8.2,7.4,4.6,0.0,38134,50325,8600000US89019,89019,41.8,46.1,12.0
2,4011.0,4.7,10.3,17.2,10.5,22.0,17.9,9.1,7.6,0.4,0.3,40700,47436,8600000US89060,89060,42.7,49.0,8.3
3,2214.0,3.2,3.6,8.1,13.3,13.6,26.3,9.6,13.9,6.5,2.0,54708,69133,8600000US89061,89061,28.2,49.5,22.4
4,724.0,3.9,4.8,3.3,14.6,18.1,16.7,8.3,13.7,0.4,16.2,57500,149584,8600000US89439,89439,26.6,43.1,30.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1763,386.0,3.4,12.7,16.8,3.4,10.9,20.7,12.4,7.0,7.5,5.2,51964,70068,8600000US96146,96146,36.3,44.0,19.7
1764,272.0,11.8,5.1,3.7,8.5,24.3,16.2,11.8,14.0,4.8,0.0,41875,59204,8600000US96148,96148,29.1,52.3,18.8
1765,11282.0,5.8,6.9,10.7,15.0,12.0,18.1,10.8,12.5,5.5,2.7,49293,65812,8600000US96150,96150,38.4,40.9,20.7
1767,7207.0,3.5,2.4,4.9,7.8,10.5,17.7,18.1,17.6,7.6,10.1,78206,106428,8600000US96161,96161,18.6,46.3,35.3


In [47]:
acs2015_filtered.to_csv("ACS-2015-5yr-Filtered1.csv")

#### Now onto the joining!